# 模型量化处理
#TODO:在完整介绍量化之前，我们都知道有些量化需要数据，有些则不需要，那么！对于需要数据的量化模型如何选择数据？
#TODO: GPTQ量化既然需要数据，我直接随机生成数据？

In [4]:
import os
import subprocess
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

## 使用llama.cpp量化
### 基本原理解释
#### 1、量化精度

#### 2、量化范式
### 1、量化大语言模型
> https://qwen.readthedocs.io/zh-cn/latest/quantization/llama.cpp.html

为了实现GGUF格式转换，按照下面步骤进行
#### 1、无校准量化GGUF
```python
# 首先创建GGUF文件  转换文件
python convert-hf-to-gguf.py 模型名称/地址 --outtype bf16 --outfile Qwen3-1.5B-BF16.gguf 
# 其中的 outtype后面的数据类型可以是：bf16 f32等
```
在执行上面代码之后直接将模型量化到`8 bit`：
```python
./build/bin/llama-quantize --quantize --model Qwen3-1.5B-F16.gguf --output Qwen3-1.5B-Q8_0.gguf --type Q8_0
```
`Q8_0`是一个量化预设的代号
#### 2、使用重要性矩阵量化GGUF
```python
# 首先
./llama-imatrix -m Qwen3-1.5B-F16.gguf -f calibration-text.txt --chunk 512 -o Qwen3-1.5B-imatrix.dat -ngl 80
# 
./llama-quantize --imatrix Qwen3-1.5B-imatrix.dat Qwen3-1.5B-F16.gguf Qwen3-1.5B-Q4_K_M.gguf Q4_K_M
```
#### 两种量化方式代码解释
在此之前可以使用`python convert_hf_to_gguf.py --help`了解一些这个脚本都有哪些参数，`python convert_hf_to_gguf.py --print-supported-models`直接看一下支持哪些模型进行量化，一般而言模型名称就和`config.json`中是一致的比如`Qwen2ForCausalLM`。量化开始：**首先**，在下载得到模型权重之后，在`cache_dir`里面就会有模型权重，比如说上面过程中模型权重就会保存为：`/Model/models--Qwen--Qwen2.5-1.5B-Instruct`，那么在创建以及转换文件过程中就需要将其中的 **模型名称/地址**进行替换，那么**完整带脚本**就是：`python convert_hf_to_gguf.py /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/Model/models--Qwen--Qwen2.5-1.5B-Instruct --outfile /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf --outtype bf16 --verbose`
**不过**，上面代码还是会出错，因为实际模型文件在`code/Python/DFModelCode/DF_acceralate/Model/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306`因此需要将上面的文件路径改为：`/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/Model/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306`最后输出：
```bash
...
INFO:hf-to-gguf:Model successfully exported to /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf
```
就表示成功了，而后可以通过`./build/bin/llama-quantize --help`去检查支持哪些参数，**并且进行量化**：`./build/bin/llama-quantize /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf q8_0`。不过还有一点有意思的，比如说我的模型一般都会用lora进行微调，我需要对lora微调模型进行处理，类似的：
```bash
python convert_lora_to_gguf.py /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/test-lora/ --outfile /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-BF16.gguf --outtype bf16 --verbose

./build/bin/llama-quantize /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-BF16.gguf /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-Q8_0.gguf q8_0
```
> **总结上面过程代码**：
> `python convert_hf_to_gguf.py /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/Model/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306 --outfile /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf --outtype bf16 --verbose`
> `./build/bin/llama-quantize /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf q8_0`

### 使用GGUF量化模型
```bash
pip install llama-cpp-python
# 直接去 https://github.com/abetlen/llama-cpp-python/releases 下载对应的版本 和flash-attn一样
pip install llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl
```
这样一来就安装好了就可以直接加载GGUF模型权重了（和非量化模型推理10.8s，**量化后推理：3.5s**）：
```python
from llama_cpp import Llama

llm = Llama(
    model_path="/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf",
    # lora_path= 
    n_ctx=8192,
    n_gpu_layers=-1,
    chat_format="chatml",
    verbose=False
)

messages = [
    {"role": "system", "content": "你是一个有帮助的助手，用简洁的中文回答。"},
    {"role": "user",   "content": "晚上睡不着怎么办"}
]

# 生成
response = llm.create_chat_completion(
    messages,
    max_tokens=300,
    temperature=0.75,
    top_p=0.95,
    repeat_penalty=1.1,
    stream=False  # 改成 True 即可流式输出
)

print("AI 回答：")
print(response["choices"][0]["message"]["content"])
```

#### 2、量化范式

In [8]:
import os
import subprocess
from transformers import AutoTokenizer, AutoModelForCausalLM
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

def load_model():
    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct",
                                                cache_dir= '/root/autodl-tmp/Model/', 
                                                mirror='https://hf-mirror.com')
    model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct",
                                                cache_dir= '/root/autodl-tmp/Model/', 
                                                mirror='https://hf-mirror.com')
    return model, tokenizer

def load_GGUF(lora_path=None):
    from llama_cpp import Llama

    llm = Llama(
        model_path="/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf",
        lora_path= lora_path,
        n_ctx=8192,
        n_gpu_layers=-1,
        chat_format="chatml",
        verbose=False
    )
    return llm
messages = [
    {"role": "system", "content": "你是一个有帮助的助手，用简洁的中文回答。"},
    {"role": "user",   "content": "晚上睡不着怎么办?"}
]
model, tokenizer = load_model()
model_gguf = load_GGUF()
model_gguf_lora = load_GGUF(lora_path= '/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-BF16.gguf')

llama_context: n_ctx_per_seq (8192) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (8192) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


## 使用llmcompressor进行模型量化
> `llmcompressor==0.7.1、transformers==4.55.2、torch==2.8.0+cu126、trl==0.22.2、vllm==0.11.0` 
### 原理及官方文档总结

| 方法 | 描述 | 准确性恢复 vs. 时间 |
|---|---|---|
| GPTQ | 利用二阶逐层优化来优先处理重要的权重/激活，并允许更新剩余权重 | 准确性恢复高，但运行成本更高/更慢 |
| AWQ | 使用通道级缩放更好地保留权重和激活中的重要异常值 | 比GPTQ具有更好的准确性恢复和更快的运行时间 |
| SmoothQuant | 通过将异常值折叠到权重中来平滑激活中的异常值，确保权重和激活量化模型具有更好的准确性 | 准确性恢复良好，校准时间最短；可与其他方法组合使用 |
| 就近取整(RTN) | 简单的量化技术，将每个值四舍五入到目标精度中最接近的可表示级别 | 在大多数情况下提供中等准确性恢复。计算成本低且实现速度快，适用于实时或资源受限的环境 |
| AutoRound | 通过符号梯度下降优化舍入和裁剪范围 | 与GPTQ/AWQ相比，提供领先的4位和优异的低于4位精度，运行时间比GPTQ快，与AWQ相当 |

直接对[官方](https://docs.vllm.ai/projects/llm-compressor/en/0.7.1/getting-started/compress/#apply-the-recipe)中描述进行总结，收其提供的压缩方式进行操作，直接使用llmcompressor中的压缩方式使用起来很简单，可以直接使用`oneshot`进行量化，比如说：
```python
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor import oneshot

recipe = [
    SmoothQuantModifier(smoothing_strength=0.8),
    GPTQModifier(scheme="W8A8", targets="Linear", ignore=["lm_head"]),
]
oneshot(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    dataset="open_platypus",
    recipe=recipe,
    output_dir="TinyLlama-1.1B-Chat-v1.0-INT8",
    max_seq_length=2048,
    num_calibration_samples=512,
)
```
**值得注意的是**，在使用其进行模型压缩过程中对于显存的估计大概是：`mem(1B parameters) ~= (1B parameters) * (2 bytes / parameter) = 2B bytes ~= 2Gb`，除此之外压缩模型过程中对于文本解码层（LLM部分）以及视觉编码层（Vision Layer部分）两部分压缩过程中处理存在差异，**前者**会动态地将模型逐层加载到GPU进行计算，其余部分仍保留在CPU内存中。**后者**会一次性全部加载到GPU。与文本模型不同，视觉编码层在加载到GPU前不会被拆分为独立层。对于视觉塔层规模大于文本层的模型，这种机制可能导致GPU内存瓶颈。目前（**测试的是0.7.1版本实际对于最新的也是如此**） LLM Compressor 暂不对视觉编码器进行量化处理，因量化技术在延迟/吞吐量与精度损失间的权衡通常得不偿失。比如说：
**1、四舍五入量化（Round To Nearest RTN）**  
![](https://s2.loli.net/2026/01/20/JiXhKPoFfpNdv7e.png)  
**2、GPTQ**  
![](https://s2.loli.net/2026/01/20/m2IqEYWsFKUPbkT.png)  
在llm compressor中支持量化操作  
![](https://s2.loli.net/2026/01/20/br2FjTWAOXp3nU5.png)    
对于[上述内容解释](https://github.com/vllm-project/llm-compressor/blob/main/docs/guides/compression_schemes.md?plain=1)：  

| Scheme | Description | Data required? | vLLM Hardware Compatibility |
|--------|-------------|----------------|-----------------------------|
| **[W8A8-FP8](../../examples/quantization_w8a8_fp8/README.md)** | 8位浮点（FP8）量化用于权重和激活值，通过8位算术运算实现约2倍的权重压缩。采用逐通道（channel-wise）量化将权重压缩至8位，并采用动态逐token或静态逐张量（per-tensor）量化将激活值压缩至8位。权重的缩放因子既可以按通道生成，也可以按张量生成。其中，逐通道权重量化配合动态逐token激活量化是性能最优的选项。W8A8-FP8无需校准数据集，激活值的量化在vLLM推理过程中实时进行。该方案在通用性能和压缩率方面表现良好，尤其适用于服务器端和批处理推理场景。 | 无需校准数据集，除非你采用静态逐张量激活量化。 | 支持最新的 NVIDIA GPU（Hopper 架构及之后）和最新的 AMD GPU。推荐用于计算能力 >=8.9 的 NVIDIA GPU（包括 Hopper、Ada Lovelace 和 Blackwell 架构）。 |
| **[W8A8-INT8](../../examples/quantization_w8a8_int8/README.md)** | 8位整数（INT8）量化用于权重和激活值，通过8位算术运算实现约2倍的权重压缩。使用GPTQ进行逐通道（channel-wise）量化将权重压缩至8位，并采用动态逐token量化将激活值压缩至8位。INT8权重量化既支持逐张量（per-tensor）也支持逐通道（per-channel）方式。W8A8-INT8在通用性能和压缩率方面表现良好，尤其适用于服务器端和批处理推理。激活值的量化在vLLM推理过程中执行，可选择静态或动态方式；此外，INT8激活值还支持非对称量化。W8A8-INT8有助于在高QPS场景或使用vLLM进行离线服务时提升推理速度。 | 权重量化以及静态逐张量激活量化需要校准数据集 | 支持所有NVIDIA GPU、AMD GPU、TPU、CPU及其他加速器。推荐用于计算能力 <8.9 的NVIDIA GPU（如Ampere、Turing、Volta、Pascal或更早架构）。 |
| **[W4A16](../../examples/quantization_w4a16/README.md)** | 仅将权重量化为4位整数（INT4）精度，激活值保持16位浮点（FP16）精度。W4A16可实现约3.7倍的权重压缩，但仍需使用16位算术运算。W4A16还支持非对称权重量化。该方案在内存受限、对延迟敏感的应用中提供最大压缩率，并在低QPS场景下通过更高的权重压缩带来显著加速。所附示例采用GPTQ算法以降低量化损失，但也可使用其他算法，如[AWQ](../../examples/awq/llama_example.py) 进行| 需要数据集 | 所有设备 |
| **W8A16** | 将模型权重编码为8位整数，激活值编码为16位整数。W8A16压缩相比FP32可生成更小的模型输出体积，并在具备原生8位整数计算单元的硬件上实现更快的推理速度。相比浮点运算，其功耗和内存带宽需求更低| 需要数据集 | 所有设备 |
| **NVFP4** | NVFP4 是随 NVIDIA Blackwell GPU 架构推出的4位浮点编码格式。它通过高精度缩放编码和两级微块（micro-block）缩放策略，在张量值的宽动态范围内保持数值精度。NVFP4 压缩为每个张量生成一个全局缩放因子，并为每16个元素组成的组生成局部量化缩放因子。权重和激活值均使用全局缩放因子和局部量化缩放因子进行量化。组大小固定，不可更改。 | 需要数据集 | 仅支持 NVIDIA Blackwell 架构及后续 GPU |
| [**W8A8-FP8_BLOCK**](../../examples/quantization_w8a8_fp8/fp8_block_example.py)| 采用分块量化（通常为 128×128 的块）将权重压缩至 FP8，并对激活值使用动态逐 token 组（每组 128 个 token）量化。无需校准数据集，激活量化在 vLLM 推理过程中实时执行 | 不需要数据集 | 支持最新的 NVIDIA GPU（Hopper 及之后）和最新的 AMD GPU。推荐用于计算能力 ≥8.9 的 NVIDIA GPU（包括 Hopper、Ada Lovelace 和 Blackwell 架构） |
| **[2:4 Semi-structured Sparsity](../../examples/sparse_2of4_quantization_fp8/README.md)** | U采用半结构化稀疏（如 SparseGPT）：在张量中每连续 4 个权重中，将其中 2 个置零。使用权重逐通道量化压缩至 8 位，并对激活值采用动态逐 token 量化压缩至 8 位。相比标准 W8A8-FP8，可实现更优推理性能，且评估分数几乎无下降，参考[evaluation score](https://neuralmagic.com/blog/24-sparse-llama-fp8-sota-performance-for-nvidia-hopper-gpus/). 但小型模型可能因剩余非零权重不足以还原原始分布而出现精度下降。| 需要数据集 |  支持所有 NVIDIA GPU、AMD GPU、TPU、CPU 及其他加速器。推荐用于计算能力 ≥9.0 的 GPU（Hopper 和 Blackwell 架构） |
| **Unstructured Sparsity** | 非结构化稀疏量化以无固定模式的方式将模型中的个别权重置零。与分块或通道剪枝不同，它在模型中任意位置移除贡献最小的权重，从而生成细粒度的稀疏矩阵。 |不需要数据集 | 支持所有 NVIDIA GPU、AMD GPU、TPU、CPU 及其他加速器 |

#### 代码原理解释
介绍其支持量化范式之后直接看代码操作
```python
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor import oneshot

recipe = [
    SmoothQuantModifier(smoothing_strength=0.8),
    GPTQModifier(scheme="W8A8", targets="Linear", ignore=["lm_head"]),
]
oneshot(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    dataset="open_platypus",
    recipe=recipe,
    output_dir="TinyLlama-1.1B-Chat-v1.0-INT8",
    max_seq_length=2048,
    num_calibration_samples=512,
)
```
**首先**看`oneshot`中参数操作，在代码`llmcompressor/entrypoints/oneshot.py`以及[文档](https://docs.vllm.ai/projects/llm-compressor/en/0.7.1/reference/llmcompressor/entrypoints/oneshot/?h=oneshot)中参数，着重了解：
* `model`：直接使用hf上模型名称/本地名称，对于本地需要具体到config.json位置比如说：`models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa30/`
* `cache_dir`：缓存路径：str
* `output_dir`：保存路径：str
* `log_dir`: 日志路径：str
* `dataset`：使用的数据集，str, Dataset, DatasetDict，如果在直接使用hf上数据集那么直接用名称就行，如open_platypus。

此处简单介绍其中**数据处理逻辑**
`dataset`-->`get_processed_dataset`(llmcompressor/datasets/utils.py)判断数据是不是提前被tokenized处理，**如果是**那么直接返回即可，**如果不是**直接使用`TextGenerationDataset.load_from_registry`(llmcompressor/transformers/finetune/data/base.py)处理数据
==>`TextGenerationDataset`处理数据逻辑，加载数据-->process数据（两部分数据和hf库datasets的处理逻辑是一致的），对于第一部分**加载数据处理如下**：
1、对于hf上数据直接加载==>返回
2、对于自定义的数据（也支持使用下载好的hf本地数据），使用`get_custom_datasets_from_path`(llmcompressor/transformers/finetune/data/data_helpers.py)，在这个函数中默认两个参数：path、ext（文件类型）。对于path格式要求
![](https://s2.loli.net/2026/01/20/my5v1aBonNuh9IZ.png)
最后都统一返回#TODO
**process数据处理如下**：
#TODO: 了解一下map函数处理过程 https://huggingface.co/docs/datasets/loading

### AWQ量化
[https://www.big-yellow-j.top/posts/2025/10/11/Quantized.html](https://www.big-yellow-j.top/posts/2025/10/11/Quantized.html)
### GPTQ量化
[https://www.big-yellow-j.top/posts/2025/10/11/Quantized.html](https://www.big-yellow-j.top/posts/2025/10/11/Quantized.html)
### 1、量化大语言模型

### 2、量化多模态模型
### 量化模型使用


In [9]:
from datasets import load_dataset

dataset = load_dataset("garage-bAInd/Open-Platypus", 
                       split="train",
                       cache_dir= '/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/')
print(f"{dataset[0]}")
dataset[0].keys()

{'input': '', 'output': 'To find the probability of the spinner landing on $C$, I need to subtract the probabilities of the spinner landing on $A$ and $B$ from $1$, since the sum of the probabilities of all possible outcomes is $1$. I can write this as an equation: $P(C) = 1 - P(A) - P(B)$. I know that $P(A) = \\frac{1}{3}$ and $P(B) = \\frac{5}{12}$, so I can plug those values into the equation and simplify. I get: $P(C) = 1 - \\frac{1}{3} - \\frac{5}{12} = \\frac{12}{12} - \\frac{4}{12} - \\frac{5}{12} = \\frac{3}{12}$. I can reduce this fraction by dividing the numerator and denominator by $3$, and I get: $P(C) = \\frac{1}{4}$. ', 'instruction': 'A board game spinner is divided into three parts labeled $A$, $B$  and $C$. The probability of the spinner landing on $A$ is $\\frac{1}{3}$ and the probability of the spinner landing on $B$ is $\\frac{5}{12}$.  What is the probability of the spinner landing on $C$? Express your answer as a common fraction.', 'data_source': 'MATH/PRM-800K'}


dict_keys(['input', 'output', 'instruction', 'data_source'])

In [ ]:
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor import oneshot

recipe = [
    SmoothQuantModifier(smoothing_strength=0.8),
    GPTQModifier(scheme="W8A8", targets="Linear", ignore=["lm_head"]),
]
oneshot(
    model='/root/autodl-tmp/Model/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306',#"Qwen/Qwen2.5-1.5B-Instruct",
    cache_dir= '/root/autodl-tmp/Model',
    dataset="open_platypus",
    recipe=recipe,
    output_dir="./tmp/LLMCompressor-GPTQ-W8A8-0.8",
    log_dir= './tmp/LLMCompressor-GPTQ-W8A8-0.8/log',
    
    max_seq_length=2048,
    num_calibration_samples=512,
)

In [1]:
from vllm import LLM
model = LLM("./tmp/LLMCompressor-GPTQ-W8A8-0.8")

/root/autodl-tmp/miniconda3/envs/docparse/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 01-20 21:23:50 [__init__.py:216] Automatically detected platform cuda.
INFO 01-20 21:23:53 [utils.py:233] non-default args: {'disable_log_stats': True, 'model': './tmp/LLMCompressor-GPTQ-W8A8-0.8'}
INFO 01-20 21:23:53 [model.py:547] Resolved architecture: Qwen2ForCausalLM
INFO 01-20 21:23:53 [model.py:1510] Using max model len 32768


2026-01-20 21:23:56,752	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 01-20 21:23:56 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=233690) INFO 01-20 21:23:57 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=233690) INFO 01-20 21:23:57 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='./tmp/LLMCompressor-GPTQ-W8A8-0.8', speculative_config=None, tokenizer='./tmp/LLMCompressor-GPTQ-W8A8-0.8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=compressed-tensors, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_parser=''),

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  3.55it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  3.52it/s]
(EngineCore_DP0 pid=233690) 


(EngineCore_DP0 pid=233690) INFO 01-20 21:23:59 [default_loader.py:267] Loading weights took 0.44 seconds
(EngineCore_DP0 pid=233690) INFO 01-20 21:24:00 [gpu_model_runner.py:2653] Model loading took 2.1034 GiB and 0.652273 seconds
(EngineCore_DP0 pid=233690) INFO 01-20 21:24:06 [backends.py:548] Using cache directory: /root/.cache/vllm/torch_compile_cache/6f84cace6e/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=233690) INFO 01-20 21:24:06 [backends.py:559] Dynamo bytecode transform time: 6.68 s
(EngineCore_DP0 pid=233690) INFO 01-20 21:24:09 [backends.py:164] Directly load the compiled graph(s) for dynamic shape from the cache, took 2.238 s
(EngineCore_DP0 pid=233690) INFO 01-20 21:24:11 [monitor.py:34] torch.compile takes 6.68 s in total
(EngineCore_DP0 pid=233690) INFO 01-20 21:24:12 [gpu_worker.py:280] The init_snapshot total_memory: 23.55 GiB The non_kv_cache_memory: 3.78 GiB
(EngineCore_DP0 pid=233690) INFO 01-20 21:24:12 [gpu_worker.py:304] Available KV cache me

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:01<00:00, 37.61it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 31.73it/s]


(EngineCore_DP0 pid=233690) INFO 01-20 21:24:16 [gpu_model_runner.py:3480] Graph capturing finished in 4 secs, took 0.65 GiB
(EngineCore_DP0 pid=233690) INFO 01-20 21:24:16 [core.py:210] init engine (profile, create kv cache, warmup model) took 16.13 seconds
INFO 01-20 21:24:17 [llm.py:306] Supported_tasks: ['generate']


In [3]:
from vllm import SamplingParams
# output = model.generate("晚上睡不着怎么办?")
# output[0].outputs[0].text
sampling_params = SamplingParams(
    temperature=0.8,           # 温度：值越高输出越随机
    top_p=0.9,                # 核采样：控制词汇多样性
    top_k=50,                 # Top-k采样
    max_tokens=1024,          # 最大生成长度
    frequency_penalty=0.1,    # 频率惩罚：减少重复内容
    presence_penalty=0.1,     # 存在惩罚：鼓励新话题
    repetition_penalty=1.1,   # 重复惩罚（某些模型支持）
)

output = model.generate("晚上睡不着怎么办?", sampling_params)
print(output[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.35s/it, est. speed input: 4.45 toks/s, output: 226.18 toks/s]

可以吃点什么?

入睡困难，睡眠质量差是一个普遍的问题，可能由多种因素引起。在寻找解决方案之前，先了解自己的生活习惯和健康状况是非常重要的。这里有一些建议可以帮助改善睡眠质量和提高睡眠质量：

1. **保持规律的作息时间**：每天尽量在同一时间上床睡觉和起床，即使是周末也不例外。

2. **创建良好的睡眠环境**：确保你的卧室安静、黑暗、适宜的温度（大约在60-67华氏度）。使用舒适的床上用品也很重要。

3. **避免咖啡因和酒精**：尤其是在睡前几小时内，尽量不要摄入咖啡因或酒精，因为它们都可能影响你的睡眠质量。

4. **放松身心**：尝试在睡前进行一些放松活动，如深呼吸练习、瑜伽、冥想或者热水浴等。

5. **减少屏幕时间**：手机、电脑和电视发出的蓝光会干扰你的生物钟并阻止褪黑素的产生，这可能会使你难以入睡。

6. **限制白天小憩**：如果你需要午睡，请控制在30分钟以内，并且最好不在下午晚些时候进行，以防止夜间入睡困难。

如果上述方法都不能有效改善你的睡眠问题，建议咨询医生或专业人士的意见。他们可以通过进一步的评估来确定是否有其他潜在的原因导致了你的失眠症状，并提供相应的治疗方案。

请记住，每个人的情况都是独特的，找到适合自己的方法至关重要。希望以上信息对你有所帮助！


## 模型微调
> https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb

https://github.com/shangxiaaabb/ProjectCode/blob/main/code/Python/DFModelCode/DF_acceralate/GRPO_training.py

## 模型加载

In [ ]:
import os
import subprocess
from transformers import AutoTokenizer, AutoModelForCausalLM
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

def load_model():
    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct",
                                                cache_dir= '/root/autodl-tmp/Model/', 
                                                mirror='https://hf-mirror.com')
    model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct",
                                                cache_dir= '/root/autodl-tmp/Model/', 
                                                mirror='https://hf-mirror.com')
    return model, tokenizer

def load_GGUF(lora_path=None):
    from llama_cpp import Llama

    llm = Llama(
        model_path="/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf",
        lora_path= lora_path,
        n_ctx=8192,
        n_gpu_layers=-1,
        chat_format="chatml",
        verbose=False
    )
    return llm
messages = [
    {"role": "system", "content": "你是一个有帮助的助手，用简洁的中文回答。"},
    {"role": "user",   "content": "晚上睡不着怎么办?"}
]
model, tokenizer = load_model()
model_gguf = load_GGUF()
model_gguf_lora = load_GGUF(lora_path= '/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-BF16.gguf')

In [9]:
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

可以试试深呼吸、听轻音乐、喝杯热牛奶或泡个热水澡等方法。如果经常失眠，建议咨询医生。<|im_end|>


In [10]:
response = model_gguf.create_chat_completion(
    messages,
    max_tokens=300,
    temperature=0.75,
    top_p=0.95,
    repeat_penalty=1.1,
    stream=False  # 改成 True 即可流式输出
)

print("AI 回答：")
print(response["choices"][0]["message"]["content"])

AI 回答：
可尝试用热水泡脚，听听轻音乐，或者看看电视。
